In [1]:
import os
import torch

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

/home/ubuntu/qlora/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ubuntu/qlora/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: CUDA runtime path found: /usr/local/cuda-11.8/lib64/libcudart.so.11.0
CUDA SETUP: Loading binary /home/ubuntu/qlora/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/ubuntu/qlora/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: No GPU detected! Check your CUDA paths. Proceeding to load CPU-only library...
  warn(msg)


In [2]:
peft_model_id = './model_file/qlora_PoC_12.8b_50steps/checkpoint-50'

peft_config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_use_double_quant = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path, 
                                             quantization_config=bnb_config, 
                                             device_map={"" : 0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

In [6]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (attention): GPTNeoXAttention(
              (rotary_emb): RotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=5120, out_features=15360, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=15360, bias=False)
                

In [5]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors = 'pt',
            return_token_type_ids = False
        ).to('cuda'),
        max_new_tokens = 256,
        early_stopping = True,
        do_sample = True,
        eos_token_id = 2
    )
    print(tokenizer.decode(gened[0]))

In [4]:
%%time
gen('몸에 단백질이 부족하면 나타나는 현상은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 몸에 단백질이 부족하면 나타나는 현상은?

### 답변: 단백질은 우리 몸에 필수 영양소인데, 체내에 부족하게 되면 탈모나 근육 약화 등이 발생할 수 있습니다. 단백질이 부족할 경우 나타나는 증상 및 식단을 확인해보겠습니다. 1. 단백질 결핍 증상1) 피부가 거칠어지고 건조해진다. 2) 탈모 혹은 흰머리가 많아진다. 3) 자주 어지러워진다. 4) 몸이 자주 피로하다.5) 면역력이 떨어져 질병에 취약해진다.6) 자주 배고프다.7) 손톱이 약해진다.8) 몸이 자주 근질거린다. 단백질이 부족할 경우 겪을 수 있는 증상이니, 혹시라도 위와 같은 증상이 나타난다면 단백질이 부족하진 않은지 체크해 보시길 바랍니다. 3. 보충이 필요한 단백질 식단 1) 두부는 100g당 79.4kcal로 굉장히 낮은 칼로리를 가지고 있으므로 식단에 자주 포함 시키시는 것이 좋습니다. 두부를 이용한 두부 유부초밥을 만드실 수 있는데, 유부초밥에 두부를 으깨 넣고 간장/참기름/다진 마늘 등을 넣고 조물조물 해주시면 됩니다. 만약, 두부에 고기 식감이 있으면 햄을
CPU times: user 50.2 s, sys: 340 ms, total: 50.5 s
Wall time: 50.5 s
